In [71]:
# ORM

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, VARCHAR, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
import time
import json

class DB:
    # 테이블 구조 정의
    base = declarative_base()
    jse = json.encoder.JSONEncoder()

    class Article(base):
        __tablename__='article_table'
        a_aid = Column(Integer, primary_key=True, autoincrement=True, nullable=False)
        a_title = Column(VARCHAR(80))
        a_uploader = Column(VARCHAR(40))
        a_timestamp = Column(DateTime)
        a_article = Column(VARCHAR(6000))

    class Reply(base):
        __tablename__='reply_table'
        r_rid = Column(Integer, primary_key=True, autoincrement=True, nullable=False)
        r_aid = Column(Integer, nullable=False)
        r_reply = Column(VARCHAR(600))
        r_uploader = Column(VARCHAR(40))
        r_timestamp = Column(DateTime)

    def time_stamp(self):
        return time.strftime("%Y-%m-%d %H:%M:%S")
    
    def temp_add(self,jsonBodyData):
        jsonData = json.loads(jsonBodyData)
        self.session.add(self.Article(a_title=jsonData['title'],
                                 a_uploader=jsonData['uploader'],
                                 a_timestamp=self.time_stamp(),
                                 a_article=jsonData['article']))
        
    def temp_search(self):
        q = self.session.query(self.Article)
        print("?")
        for l in q:
            print(l.a_aid,l.a_title,l.a_uploader,l.a_timestamp,l.a_article)
    
    # 엔진 생성
    def create_engine_from_port(self,port,dbProtocol,dbPwd,dbContainerAddress):
        return create_engine("%s://%s@%s:%s"%(dbProtocol,dbPwd,dbContainerAddress,port), echo=False)
    
    # 세션 생성
    def create_session(self,engine):
        Session = sessionmaker(autocommit=False, autoflush=True, bind=engine)
        session = Session()
        return session
    
    # DB 내에 인턴 명부 테이블 생성
    def create_DB(self):
        self.base.metadata.create_all(self.engine)
        self.session.commit()
    
    
    # 인스턴스 생성 시 localhost 포트에 대한 엔진, 세션 인스턴스를 생성
    def __init__(self,port,dbProtocol='postgresql',dbPwd='postgres:postgrespw',dbContainerAddress='localhost'):    
        self.engine=self.create_engine_from_port(port,dbProtocol,dbPwd,dbContainerAddress)
        self.session=self.create_session(self.engine)


In [72]:
db=DB(49153) # local host의 postgresql 의 컨테이너 localhost 포트 번호를 입력 받아 DB 인스턴스 생성

In [73]:
db.create_DB() # 처음으로 컨테이너를 실행할 때, 혹은 볼륨을 삭제했을 때 실행시켜줘야 함

In [74]:
testArticlePost = {}
testArticlePost['title'] = "제목 테스트"
testArticlePost['uploader'] = "글 테스터"
testArticlePost['article'] = "대충 이런 식으로 본문이 들어감\n잘 보이나?"
# testArticlePost['timestamp'] = time.strftime("%Y-%m-%d %H:%M") # timestamp는 DB에서 처리

jbd = db.jse.encode(testArticlePost)

In [75]:
jbd

'{"title": "\\uc81c\\ubaa9 \\ud14c\\uc2a4\\ud2b8", "uploader": "\\uae00 \\ud14c\\uc2a4\\ud130", "article": "\\ub300\\ucda9 \\uc774\\ub7f0 \\uc2dd\\uc73c\\ub85c \\ubcf8\\ubb38\\uc774 \\ub4e4\\uc5b4\\uac10\\n\\uc798 \\ubcf4\\uc774\\ub098?"}'

In [84]:
db.temp_add(jbd)

In [85]:
db.temp_search()

?
7 제목 테스트 글 테스터 2023-01-10 14:05:11 대충 이런 식으로 본문이 들어감
잘 보이나?
8 제목 테스트 글 테스터 2023-01-10 14:05:16 대충 이런 식으로 본문이 들어감
잘 보이나?
9 제목 테스트 글 테스터 2023-01-10 14:05:18 대충 이런 식으로 본문이 들어감
잘 보이나?
10 제목 테스트 글 테스터 2023-01-10 14:05:19 대충 이런 식으로 본문이 들어감
잘 보이나?
11 제목 테스트 글 테스터 2023-01-10 14:05:21 대충 이런 식으로 본문이 들어감
잘 보이나?
